In [138]:
from sympy import *
import numpy as np
from timeit import default_timer as timer
from sympy.printing.theanocode import theano_function
from numba import vectorize, float64, autojit

## Obtaining a compiled function from a symbolic expression

In [139]:
x = symbols("x")
f = cos(x)*sin(x)**2 + sin(4*(x - x**2*sin(x)))
lf  = lambdify(x,f,"numpy")
npf = np.vectorize(lambdify(x,f,"numpy"))
thf = theano_function([x], [f], dims={x:1})

@vectorize([float64(float64)], target="cpu")
def nmf(x):
    return lf(x)

@autojit
def cf(x):
    return lf(x)

@vectorize([float64(float64)], target="cpu")
def cvf(x):
    return cf(x)


In [140]:
npf([1,2,3,4,5])

array([ 0.97503935, -0.60654201,  0.57466439,  0.62509067,  0.60044611])

In [141]:
thf([1,2,3,4,5])

array([ 0.97503935, -0.60654201,  0.57466439,  0.62509067,  0.60044611])

In [142]:
nmf([1,2,3,4,5])

array([ 0.97503935, -0.60654201,  0.57466439,  0.62509067,  0.60044611])

In [143]:
a = np.random.random(1000)
%timeit npf(a)

100 loops, best of 3: 5.4 ms per loop


In [144]:
%timeit nmf(a)

100 loops, best of 3: 7.61 ms per loop


In [145]:
%timeit thf(a)

The slowest run took 5.12 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 55.9 µs per loop


In [146]:
%timeit cvf(a)

100 loops, best of 3: 9.42 ms per loop


If too simple, performance improve is not so large due to overheads

In [123]:
f = 2*x + x**2
lf  = lambdify(x,f,"numpy")
npf = np.vectorize(lambdify(x,f,"numpy"))
thf = theano_function([x], [f], dims={x:1})

In [124]:
a = np.random.random(1000)
%timeit npf(a)

1000 loops, best of 3: 392 µs per loop


In [125]:
%timeit thf(a)

The slowest run took 7.67 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 12.8 µs per loop


## Observe that having the analytic solution we can:

- compile it to run fast
- split the computation into as many tasks as we want

but with numerical solutions CANNOT do either since:

- `odeint` is a python function
- the process is intrinsically sequential. at most we can parallelize the n computations in each time step required for each method.
